# Case Genius — Gemini Human‑in‑the‑Loop, Game‑Theoretic Analysis

Consultant-style | Ambiguity-aware | UI-driven

In [ ]:

import json, time
from typing import Dict, List
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:

import google.generativeai as genai

GEMINI_API_KEY = "PASTE_YOUR_KEY"
GEMINI_MODEL = "gemini-2.5-flash"

genai.configure(api_key=GEMINI_API_KEY)
_model = genai.GenerativeModel(GEMINI_MODEL)

def llm_generate(prompt: str) -> str:
    try:
        resp = _model.generate_content(prompt)
        return getattr(resp, "text", "") or ""
    except Exception as e:
        return f"[LLM ERROR] {e}"


In [ ]:

AGENTS = {
    "Analyst": "Decompose problem, identify levers, structure analysis",
    "Skeptic": "Attack assumptions, surface risks and downsides",
    "Optimist": "Highlight upside, growth paths, non-obvious opportunities",
    "Regulator": "Check feasibility, constraints, ethics, compliance"
}


In [ ]:

STATE = {
    "problem": "",
    "assumptions": [],
    "agent_outputs": {},
    "ambiguities": [],
    "clarifications": {},
    "final_report": ""
}


In [ ]:

top_k = widgets.IntSlider(
    value=2, min=1, max=4, step=1,
    description="Top-K Touchpoints",
    style={"description_width": "initial"}
)

display(top_k)


In [ ]:

problem_box = widgets.Textarea(
    placeholder="Describe the business / strategy problem…",
    layout=widgets.Layout(width="100%", height="120px")
)

lock_problem_btn = widgets.Button(description="Lock Problem", button_style="primary")

def lock_problem(_):
    STATE["problem"] = problem_box.value
    clear_output()
    print("✅ Problem locked")
    display(top_k)

lock_problem_btn.on_click(lock_problem)
display(problem_box, lock_problem_btn)


In [ ]:

gen_assumptions_btn = widgets.Button(description="Generate Assumptions")

def gen_assumptions(_):
    prompt = f"""
You are a consultant.
List the key assumptions required to analyze this problem.
Problem:
{STATE['problem']}
"""
    text = llm_generate(prompt)
    STATE["assumptions"] = [a.strip() for a in text.split("\n") if a.strip()]
    clear_output()
    print("Assumptions:")
    for a in STATE["assumptions"][:top_k.value]:
        print("-", a)
    display(gen_assumptions_btn)

gen_assumptions_btn.on_click(gen_assumptions)
display(gen_assumptions_btn)


In [ ]:

run_agents_btn = widgets.Button(description="Run Agents")

def run_agents(_):
    STATE["agent_outputs"] = {}
    clear_output()
    for agent, role in AGENTS.items():
        prompt = f"""
You are acting as a {agent}.
Role: {role}

Problem:
{STATE['problem']}

Assumptions:
{STATE['assumptions']}
"""
        out = llm_generate(prompt)
        STATE["agent_outputs"][agent] = out
        print(f"=== {agent} ===")
        print(out[:1200])
        print("\n")
    display(run_agents_btn)

run_agents_btn.on_click(run_agents)
display(run_agents_btn)


In [ ]:

detect_ambiguity_btn = widgets.Button(description="Detect Ambiguities")

def detect_ambiguity(_):
    prompt = f"""
Identify ambiguities, unclear variables, or decision forks in the analysis below.
Return a short list.

{STATE['agent_outputs']}
"""
    text = llm_generate(prompt)
    STATE["ambiguities"] = [x.strip() for x in text.split("\n") if x.strip()]
    clear_output()
    print("Ambiguities (Top-K):")
    for a in STATE["ambiguities"][:top_k.value]:
        print("-", a)
    display(detect_ambiguity_btn)

detect_ambiguity_btn.on_click(detect_ambiguity_btn)
display(detect_ambiguity_btn)


In [ ]:

clarification_box = widgets.Textarea(
    placeholder="Provide clarifications / decisions…",
    layout=widgets.Layout(width="100%", height="120px")
)

save_clarification_btn = widgets.Button(description="Save Clarification")

def save_clarification(_):
    STATE["clarifications"][time.time()] = clarification_box.value
    clear_output()
    print("Clarification saved.")
    display(clarification_box, save_clarification_btn)

save_clarification_btn.on_click(save_clarification_btn)
display(clarification_box, save_clarification_btn)


In [ ]:

generate_report_btn = widgets.Button(description="Generate Final Report", button_style="success")

def generate_report(_):
    prompt = f"""
You are a senior consultant.
Produce a concise, executive-ready report.

Problem:
{STATE['problem']}

Assumptions:
{STATE['assumptions']}

Agent Views:
{STATE['agent_outputs']}

Clarifications:
{STATE['clarifications']}
"""
    STATE["final_report"] = llm_generate(prompt)
    clear_output()
    print(STATE["final_report"])
    display(generate_report_btn)

generate_report_btn.on_click(generate_report_btn)
display(generate_report_btn)


In [ ]:

export_btn = widgets.Button(description="Export STATE to JSON")

def export_state(_):
    with open("case_genius_state.json", "w") as f:
        json.dump(STATE, f, indent=2)
    print("State exported to case_genius_state.json")

export_btn.on_click(export_state)
display(export_btn)
